In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as pl
from matplotlib import rcParams
import time
from datetime import timedelta
import scanpy as sc
sc.settings.verbosity = 3  # verbosity: errors (0), warnings (1), info (2), hints (3)
print(sc.logging.print_versions())
import os
dirname = os.getcwd()
print(dirname)

The `sinfo` package has changed name and is now called `session_info` to become more discoverable and self-explanatory. The `sinfo` PyPI package will be kept around to avoid breaking old installs and you can downgrade to 0.3.2 if you want to use it without seeing this message. For the latest features and bug fixes, please install `session_info` instead. The usage and defaults also changed slightly, so please review the latest README at https://gitlab.com/joelostblom/session_info.
-----
anndata     0.7.8
scanpy      1.8.2
sinfo       0.3.4
-----
PIL                         9.0.1
asttokens                   NA
backcall                    0.2.0
beta_ufunc                  NA
binom_ufunc                 NA
bottleneck                  1.3.4
cffi                        1.15.0
cloudpickle                 2.0.0
colorama                    0.4.4
cycler                      0.10.0
cython_runtime              NA
cytoolz                     0.11.0
dask                        2022.02.1
dateutil    

In [6]:
from sklearn.metrics import silhouette_score

def silhouette_coeff_ASW(adata, method_use='raw',save_dir='', save_fn='', percent_extract=0.8):
    random.seed(0)
    asw_fscore = []
    asw_bn = []
    asw_bn_sub = []
    asw_ctn = [] 
    iters = []
    for i in range(20):
        iters.append('iteration_'+str(i+1))
        rand_cidx = np.random.choice(adata.obs_names, size=int(len(adata.obs_names) * percent_extract), replace=False)
#         print('nb extracted cells: ',len(rand_cidx))
        adata_ext = adata[rand_cidx,:]
        asw_batch = silhouette_score(adata_ext.X, adata_ext.obs['batch'])
        asw_celltype = silhouette_score(adata_ext.X, adata_ext.obs['cell_type'])
        min_val = -1
        max_val = 1
        asw_batch_norm = (asw_batch - min_val) / (max_val - min_val)
        asw_celltype_norm = (asw_celltype - min_val) / (max_val - min_val)
        
        fscoreASW = (2 * (1 - asw_batch_norm)*(asw_celltype_norm))/(1 - asw_batch_norm + asw_celltype_norm)
        asw_fscore.append(fscoreASW)
        asw_bn.append(asw_batch_norm)
        asw_bn_sub.append(1-asw_batch_norm)
        asw_ctn.append(asw_celltype_norm)
    
#     iters.append('median_value')
#     asw_fscore.append(np.round(np.median(fscoreASW),3))
#     asw_bn.append(np.round(np.median(asw_batch_norm),3))
#     asw_bn_sub.append(np.round(1 - np.median(asw_batch_norm),3))
#     asw_ctn.append(np.round(np.median(asw_celltype_norm),3))
    asw_bn_mean = np.mean(asw_bn)
    asw_bn_sub_mean = np.mean(asw_bn_sub)
    asw_ctn_mean = np.mean(asw_ctn)
    asw_fscore_mean = np.mean(asw_fscore)
    asw_mean = [method_use, asw_bn_mean, asw_bn_sub_mean, asw_ctn_mean,asw_fscore_mean]
    df = pd.DataFrame({'asw_batch_norm':asw_bn, 'asw_batch_norm_sub': asw_bn_sub,
                       'asw_celltype_norm': asw_ctn, 'fscore':asw_fscore,
                       'method_use':np.repeat(method_use, len(asw_fscore))})
    df.to_csv(os.path.join(save_dir + save_fn + '.csv'))
    print('Save output of pca in: ',save_dir)
    print(df.values.shape)
    print(df.keys())
    return df,asw_mean
        

In [3]:
def createAnnData(data_dir,myDatafn):
    
    myData = pd.read_csv(os.path.join(data_dir, myDatafn),header=0, index_col=0)
    bex = ['batch','Batch','Batchlb','batchlb','BATCH']
    ib = np.isin(myData.keys(), bex)
    cex = ['celltype','CellType','cell_type','Cell_Type','ct']
    ict = np.isin(myData.keys(), cex)
    adata = sc.AnnData(myData.values[:,0:9])
    adata.obs_names = myData.index
    adata.obs['batch'] = myData.values[:, np.where(ib)[0][0]]  # factor function in R
    adata.obs['cell_type'] = myData.values[:, np.where(ict)[0][0]]
    print(adata)
    return adata

In [9]:
data_dir = 'E:/mymodel/methodcompare/datasets/mouse_neuron_pca/'
save_dir = 'E:/mymodel/methodcompare/results/mouse neuron/'
print(save_dir)
if not os.path.exists(save_dir+'asw_output/'): os.makedirs(os.path.join(save_dir,'asw_output/'))
fls = [ f for f in os.listdir(data_dir) if f.endswith("_pca.csv") & os.path.isfile(os.path.join(data_dir,f)) ]
fls


E:/mymodel/methodcompare/results/mouse neuron/


['mouse_bbknn_pca.csv',
 'mouse_combat_pca.csv',
 'mouse_harmony_pca.csv',
 'mouse_limma_pca.csv',
 'mouse_mnn_pca.csv',
 'mouse_quantnorm_pca.csv',
 'mouse_raw_pca.csv',
 'mouse_scAEQN_pca.csv',
 'mouse_scanorama_pca.csv',
 'mouse_scbatch_pca.csv',
 'mouse_scvi_pca.csv']

In [10]:
import random
final_ls = []
for f in fls: 
    method_use = f[6:-8]
    print('Extract asw for ', method_use)
    save_fn = 'asw_' + method_use
    print(method_use)
    adata = createAnnData(data_dir,f)
    asw_val, asw_mean = silhouette_coeff_ASW(adata, method_use, os.path.join(save_dir,'asw_output/'), 
                                          save_fn, percent_extract=0.8)
    final_ls.append(asw_mean)
    

result = pd.DataFrame(final_ls, columns=['method','asw_bn_mean','asw_bn_sum_mean','asw_cln_mean','asw_fscore_mean'])
result.index = result['method']
result =  result.drop('method',axis=1)
result.to_csv(os.path.join(save_dir, 'asw_output/', 'final_asw_summary.csv'))

Extract asw for  bbknn
bbknn
AnnData object with n_obs × n_vars = 610 × 9
    obs: 'batch', 'cell_type'
Save output of pca in:  E:/mymodel/methodcompare/results/mouse neuron/asw_output/
(20, 5)
Index(['asw_batch_norm', 'asw_batch_norm_sub', 'asw_celltype_norm', 'fscore',
       'method_use'],
      dtype='object')
Extract asw for  combat
combat
AnnData object with n_obs × n_vars = 610 × 9
    obs: 'batch', 'cell_type'
Save output of pca in:  E:/mymodel/methodcompare/results/mouse neuron/asw_output/
(20, 5)
Index(['asw_batch_norm', 'asw_batch_norm_sub', 'asw_celltype_norm', 'fscore',
       'method_use'],
      dtype='object')
Extract asw for  harmony
harmony
AnnData object with n_obs × n_vars = 610 × 9
    obs: 'batch', 'cell_type'
Save output of pca in:  E:/mymodel/methodcompare/results/mouse neuron/asw_output/
(20, 5)
Index(['asw_batch_norm', 'asw_batch_norm_sub', 'asw_celltype_norm', 'fscore',
       'method_use'],
      dtype='object')
Extract asw for  limma
limma
AnnData object wi